## Notebook to compute total demand by TAZ for the truck mode

In [ ]:
# Notebook to compute total demand by TAZ for the truck mode

import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
%matplotlib notebook

### User input required: Specify paths to input and output directories in config.py file

In [ ]:
%run "S:/jupyter_notebooks/config.py"

### Specify scenario to be used in the current run of this notebook

In [ ]:
# Specify scenario to be used in the current run of this notebook
# 
home_dir = base_scenario_dir

### User input required: Name of CSV output file

In [ ]:
# Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'taz_truck_report_base.csv'

In [ ]:
taz_shapefile_base_dir = reference_data_dir + 'canonical_TAZ_shapefile/'

In [ ]:
# trip_tables directory - this really "should" be a subdirectory of the base directory, but is isn't currently.
# The real McCoy - where things should go, and will eventually go
tt_dir = home_dir + 'out/'

In [ ]:
# Function to open OMX trip_tables files
def open_trip_tables(tt_dir):
    tt_am = tt_dir + 'AfterSC_Final_AM_Tables.omx'
    tt_md = tt_dir + 'AfterSC_Final_MD_Tables.omx'
    tt_pm = tt_dir + 'AfterSC_Final_PM_Tables.omx'
    tt_nt = tt_dir + 'AfterSC_Final_NT_Tables.omx'
    tt_omxs = { 'am' : omx.open_file(tt_am,'r'),
                'md' : omx.open_file(tt_pm,'r'),
                'pm' : omx.open_file(tt_pm,'r'),
                'nt' : omx.open_file(tt_nt,'r') 
              }   
    return tt_omxs
# end_def open_trip_tables

In [ ]:
# Function to load trip tables as NumPy arrays from OMX files
def load_trip_tables(tt_omxs, modes):
    periods = ['am', 'md', 'pm', 'nt']
    retval  = {'am' : {}, 'md' : {}, 'pm' : {}, 'nt' : {} }
    for period in periods:
        for mode in modes:
            temp = tt_omxs[period][mode]
            retval[period][mode] = np.array(temp)
        # end_for
    # end_for
    return retval
# end_def 

### Load trip tables for truck mode

In [ ]:
# *** Load trip tables for the truck mode
# 
tt_omxs = open_trip_tables(tt_dir)
temp = load_trip_tables(tt_omxs, \
     ['Heavy_Truck', 'Heavy_Truck_HazMat', 'Medium_Truck', 'Medium_Truck_HazMat', 'Light_Truck'])                   

In [ ]:
# Mapping from TAZ-ID to OMX index for the 4 periods (these *should* be the same)
taz_to_omxid_am = tt_omxs['am'].mapping('ID')
taz_to_omxid_am = tt_omxs['md'].mapping('ID')
taz_to_omxid_pm = tt_omxs['pm'].mapping('ID')
taz_to_omxid_nt = tt_omxs['nt'].mapping('ID')

In [ ]:
# We'll assume that the mapping from TAZ ID to OMX ID doesn't vary by time period.
# We'll use the AM mapping as _the_ mapping for all time periods, pending confirmation.
# 
# TBD: Insert "sanity check" that the 4 mappings on "ID" are identical.
#
taz_to_omxid = taz_to_omxid_am

In [ ]:
# Compute total Heavy, Medium, Light, and total truck demand from each TAZ
#
heavy_temp = temp['am']['Heavy_Truck'] + temp['md']['Heavy_Truck'] + temp['pm']['Heavy_Truck'] + temp['nt']['Heavy_Truck']
heavy_demand = heavy_temp.sum(axis=1)
heavy_haz_temp = temp['am']['Heavy_Truck_HazMat'] + temp['md']['Heavy_Truck_HazMat'] + temp['pm']['Heavy_Truck_HazMat'] + temp['nt']['Heavy_Truck_HazMat']
heavy_haz_demand = heavy_haz_temp.sum(axis=1)
heavy_demand_total = heavy_demand + heavy_haz_demand
#
medium_temp = temp['am']['Medium_Truck'] + temp['md']['Medium_Truck'] + temp['pm']['Medium_Truck'] + temp['nt']['Medium_Truck']
medium_demand = medium_temp.sum(axis=1)
medium_haz_temp = temp['am']['Medium_Truck_HazMat'] + temp['md']['Medium_Truck_HazMat'] + temp['pm']['Medium_Truck_HazMat'] + temp['nt']['Medium_Truck_HazMat']
medium_haz_demand = medium_haz_temp.sum(axis=1)
medium_demand_total = medium_demand + medium_haz_demand
#
light_temp = temp['am']['Light_Truck'] + temp['md']['Light_Truck'] + temp['pm']['Light_Truck'] + temp['nt']['Light_Truck']
light_demand_total = light_temp.sum(axis=1)
# 
total_truck_demand = heavy_demand_total + medium_demand_total + light_demand_total

In [ ]:
# Build dataframes of Heavy, Mediu, Light, and total truck demand from each TAZ
# Set each data frame's index to the omxid of each row, i.e., its index
heavy_df = pd.DataFrame(heavy_demand_total, columns=['heavy_truck'])
heavy_df['omxid'] = heavy_df.index
heavy_df.set_index('omxid')
#
medium_df = pd.DataFrame(medium_demand_total, columns=['medium_truck'])
medium_df['omxid'] = medium_df.index
medium_df.set_index('omxid')
#
light_df = pd.DataFrame(light_demand_total, columns=['light_truck'])
light_df['omxid'] = light_df.index
light_df.set_index('omxid')
#
truck_df = pd.DataFrame(total_truck_demand, columns=['truck_total'])
truck_df['omxid'] = truck_df.index
truck_df.set_index('omxid')

In [ ]:
# Join the 4 dataframes into a single data frame
temp1_df = pd.merge(left=truck_df, right=heavy_df, on="omxid")
temp2_df = pd.merge(left=temp1_df, right=medium_df, on="omxid")
total_truck_trips_df = pd.merge(temp2_df, light_df, on="omxid")

In [ ]:
# Load the candidate canonical TAZ shapefile as a geopands dataframe.
# N.B. Use shapefile in WGS84 SRS.
#
taz_shapefile = taz_shapefile_base_dir + 'candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp'
taz_gdf = gp.read_file(taz_shapefile)
taz_gdf.set_index("id")

In [ ]:
# Add a 'omxid' column to the TAZ geodataframe, in prep for joining with the total trips dataframes.
# ==> This also can be done earlier.
taz_gdf['omxid'] = taz_gdf.apply(lambda row: taz_to_omxid[row.id], axis=1)

In [ ]:
# Join the shapefile geodataframe to the total trips dataframe on 'omxid'
joined_df = taz_gdf.join(total_truck_trips_df.set_index('omxid'), on='omxid')

### Export report output as CSV file

In [ ]:
# Export the useful columns of data in the 'joined_df' dataframe as a CSV file
fq_output_fn = sandbox_dir + csv_output_fn
joined_df.to_csv(fq_output_fn, sep=',', 
                columns=['id', 'town', 'state', 'truck_total', 'heavy_truck', 'medium_truck', 'light_truck'])

### Generate static and interactive maps of results

In [ ]:
# Make a static map of total auto trips by origin TAZ
joined_df.plot("truck_total", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Total Truck Trips by Origin TAZ')
plt.show()

In [ ]:
# Make an interactive map of the above
joined_df.hvplot(c='truck_total', 
                 geo=True, 
                 hover_cols=['id', 'town', 'truck_total', 'heavy_truck', 'medium_truck', 'light_truck'], 
                 clabel='Total Truck Trips', 
                 cmap='plasma',
                 frame_height=500).opts(title='Total Truck Trips by Origin TAZ')